# Linear Algebra Group Project
#### Group Members: Corey Dobbs and Emma Rasmussen 

### Problem 1
#### Part a: Write as a variational problem 
 
We are given the following for our boundary problem: 
$$ 
-u''(x) + V(x)u'(x) = f(x), x\epsilon [0,1]\\
u(0) = u(1) = 0\\
V(x) = \gamma, f(x) = 1\\
$$ 
We first start by picking a test function, $\phi(x)$. We then mupliply our initial function by this value and interate over x to get the following: 
$$ 
\quad -\int_{0}^{1} u''(x)\phi(x) \, dx + \int_{0}^{1} u'(x)V(x)\phi(x) \, dx = \int_{0}^{1} f(x)\phi(x) \, dx
$$
Focusing on the first term, we can use integration by parts to expand that to: 
$$
\left. -\phi \right|_{0}^{1} + \int_{0}^{1} u'(x)\phi'(x) \,dx 
$$
The first term of the above equation will go to zero, so getting rid of that and plugging our new expression back into our original intergral equation, we would end up with: 
$$
\quad \int_{0}^{1} u'(x)\phi'(x) \,dx + \int_{0}^{1} u'(x)V(x)\phi(x) \, dx = \int_{0}^{1} f(x)\phi(x) \, dx
$$ 
Where the terms, respecitvely, correspond to: 
$$ 
A_1[\phi(x),u(x)] + A_2[\phi(x),u(x)] = F[\phi(x)] 
$$ 

#### Part b: Expand u as a linear combination

Next, we will be expanding u as the given linear combination, $u = \sum_{i=1}^{m} u_i\phi_i(x)$. We will stop writing the x dependence of $\phi$ for simplicity, but note that it still exists. Also, we will assume that we are choosing $\phi$ from a finite dimensional basis $\{\phi_1, \phi_2, ...,\phi_m\}$:
$$
A_1[\phi_j,\sum_{i=1}^{m} u_i\phi_i] + A_2[\phi_j,\sum_{i=1}^{m} u_i\phi_i] = F[\phi_j] 
$$
Since $u_i$ is a scalar, we can pull that and the sum out and simplify to: 
$$ 
\sum_{i=1}^{m} u_i(A_1[\phi_j,\phi_i] + A_2[\phi_j,\phi_i]) = F[\phi_j] 
$$ 
In this expression, $u_i$ corresponds to $x$, $A_1[\phi_j,\phi_i] + A_2[\phi_j,\phi_i]$ corersponds to $A_{ij}$, and $F[\phi_j] $ corresponds to $b$, which shows that our problem is in the form $Ax-b$. 

#### Part c: Implement a driver routine that will return A and b given inputs n and gamma.  The matrix A should be implemented as a sparse representation in your environment.

In [3]:
import numpy as np
from scipy.sparse import csr_matrix

def driver(n, g):
    A = csr_matrix((n,n))
    

In [3]:
from scipy.sparse import csr_matrix, diags
import numpy as np

def driver( n, gamma ):
    A2 = np.zeros( (n+1,n+1) )
    diagonalsA1 = np.ones( n ) * 2 * (n+1)
    off_diagonalsA1 = np.ones( n-1 ) * -(n+1)
    A1 = diags( [diagonalsA1,off_diagonalsA1,off_diagonalsA1], [0,1,-1] )

    diagonalsA2 = np.zeros( n )
    upper_diags = np.ones( n ) * 0.5
    lower_diags = np.ones( n ) * -0.5
    A2 = gamma*diags( [diagonalsA2,upper_diags,lower_diags], [0,1,-1] )
 
    A = A1 + A2
    b = np.ones( n ) / (n+1)
    return A, b, A1, A2

A, b, A1, A2 = driver( 3, gamma=1 )
# print(A1)
# print(A2)
print(A.toarray())


[[ 8.  -3.5  0. ]
 [-4.5  8.  -3.5]
 [ 0.  -4.5  8. ]]


In [24]:
def mygmres( I, b, x0, n, A ):

    r0 = b - A @ x0
    beta = np.linalg.norm( r0, 2 )
    
    v1 = (r0 / beta).T
    v = np.zeros( (len(v1),n) )
    v[:,0] = v1

    H = np.zeros( (n+1,n) )
    w = np.zeros((n,n))

    for j in range( I ):
        w[:,j] = np.dot( A, v[:,j] )
        for i in range( j+1 ):
            H[i,j] = np.dot( w[:,j], v[:,i] )
            w[:,j] = w[:,j] - H[i,j] * v[:,i]

        H[j+1,j] = np.linalg.norm( w[:,j], 2 )

        if abs(H[j+1,j]) < 1e-8:
            break

        v[:,j+1] = w[:,j] / H[j+1,j]

    e1 = np.zeros( (n,1) )
    e1[0] = 1
    H = H[:n,]
    y = np.linalg.solve( H, beta*e1 )
    x = v @ y

    return x

A = np.array([[1,4,7],[2,9,7],[5,8,3]])
b = np.array([1,8,2]).T
x0=np.zeros(3)

ytest = mygmres( 3, b, x0, 3, A )
print(ytest)

[[ 0.12038585  0.54968971  0.82664894]
 [ 0.96308682 -0.26663002  0.03704323]
 [ 0.24077171  0.79167522 -0.5614974 ]] [[13.05797101  5.40980927 -1.56692078]
 [ 7.43353946  3.99869214  1.0668046 ]
 [ 0.          2.63182069 -4.05666316]] [[-2.18103448]
 [ 1.8362069 ]
 [-0.59482759]] [[ 1.36264332]
 [-2.15939034]
 [-1.40093667]]
